In [2]:
import pandas as pd
import numpy as np
import plotly.express as ex

In [3]:
calendar = pd.read_csv('Data/calendar.csv')
listings = pd.read_csv('Data/listings.csv')
reviews = pd.read_csv('Data/reviews.csv')

# Take a look

In [4]:
calendar.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [59]:
calendar.shape[0]

1393570

In [5]:
calendar.price.isna().sum() / calendar.shape[0]

0.32938998399793334

In [7]:
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


In [11]:
listings.shape[0]

3818

In [12]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


In [13]:
reviews.shape[0]

84849

Convert date column to datetime data type and create a new column which contains only year and month

In [15]:
calendar['date'] = pd.to_datetime(calendar.date)
calendar['year_month'] = calendar.apply(lambda x:  str(x.date.year) + '-' + str(x.date.month), axis=1)

# Analysis

## What is the percentage of occupied rooms

In [36]:
# aggregate for each month
counts = calendar.groupby('year_month').available.value_counts(normalize=True)
counts = counts.rename_axis(['date','a']).reset_index().rename(columns={'a':'available', 'available':'count'})

In [37]:
# drop 2017 column
counts = counts.drop(counts[counts.date == '2017-1'].index)

# create month column and sort by it
counts['month'] = counts.apply(lambda x : int(x.date.split('-')[1]), axis=1)
counts = counts.sort_values(by='month')

In [39]:
fig = ex.bar(counts[counts.available == 'f'], x='month', y='count', color='available', barmode='group',
             labels={"t": "Gender", "f": "Smokes"}, title='Percentage of occupied rooms', text_auto='.2p')

# specify x tick for better interpreting of the chart
months = ['January','February','March','April','May','June','July','August','September','October','November','December']
values = counts['month'].unique()
fig.update_layout(xaxis=dict(tickvals = values, ticktext = months), showlegend=False, title_x=0.5, yaxis_title='')
fig


## Prices across the year

In [40]:
# extract price without dollar sign and convert the string to float
calendar['price2'] = calendar.apply(lambda x: np.nan if pd.isna(x.price) else float(x.price[1:].replace(',','')), axis=1)

In [53]:
# aggregate for each month
counts2 = calendar.groupby('year_month').price2.mean().reset_index()
counts2.year_month = pd.to_datetime(counts2.year_month)

# drop 2017
counts2 = counts2.drop(counts2[counts2.year_month == '2017-01-01'].index)

In [56]:
# create month columns
counts2['month'] = counts2.year_month.dt.month
counts2 = counts2.sort_values(by='month')

In [58]:
fig = ex.line(counts2, x='year_month', y='price2', title='Prices across the year', text=counts2['price2'].round(1).astype(str)+'$')

# specify x tick for better interpreting of the chart
values = counts2['year_month'].unique()
fig.update_layout(xaxis=dict(tickvals = values, ticktext = months),
                  showlegend=False, title_x=0.5, xaxis_title='Date', yaxis_title='Average Price')

fig.update_traces(textposition="top right")
fig


## Most and least expensive neighborhood?

In [89]:
# select neighbourhoods with less than 10 records
cities_less_than_10 = listings.neighbourhood.value_counts()[-21:-1].index
listings.neighbourhood.isin(cities_less_than_10).sum()

103

In [90]:
# drop neighbourhoods
listings2 = listings.drop(listings[listings.neighbourhood.isin(cities_less_than_10)].index)

In [91]:
# aggregate
price_nb = listings2.groupby('neighbourhood').price2.mean().reset_index().sort_values(by='price2', ascending=False)

In [139]:
# select top and least 5 and concatenate them
top = price_nb.iloc[:5].copy()
top['type'] = 'Top'

less = price_nb.iloc[-5:].copy()
less['type'] = 'Least'

both = top.append(less)

fig = ex.bar(both, x='neighbourhood', y='price2', title='Average price for neighbourhoods', color='type', text=both['price2'].round(1).astype(str)+'$')
fig.update_layout(title_x=0.5, xaxis_title='Neighbourhood', yaxis_title='Average price')
fig

C:\Users\Aziz\AppData\Local\Temp\ipykernel_30288\4202422777.py:7: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

